In [1]:
import json
import csv
import re
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

def cosine_score(text1, text2, model):
    """
    Calculate the cosine similarity score between two texts using SentenceTransformer.

    Parameters:
    text1 (str): The first text to compare.
    text2 (str): The second text to compare.
    model: Pre-loaded SentenceTransformer model to use for embeddings.

    Returns:
    float: The cosine similarity score between the two texts.
    """
    # Encode the texts into embeddings
    embeddings1 = model.encode([text1])
    embeddings2 = model.encode([text2])

    # Calculate the cosine similarity between the embeddings
    score = cosine_similarity(embeddings1, embeddings2)

    # Return the similarity score
    return score[0][0]

def extract_test_type_and_number(name):
    """Extract the test type and number from the test name."""
    match = re.match(r"Test (\w+) #(\d+)", name)
    if match:
        test_type, test_number = match.groups()
        return test_type.lower(), test_number
    return None, None

def convert_json_to_csv(json_file, csv_file):
    # Load the JSON data
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    # Load the pre-trained SentenceTransformer model once
    print("Loading SentenceTransformer model...")
    model = SentenceTransformer('all-distilroberta-v1')
    
    # Create a dictionary to organize data by test type and number
    test_data = {}
    test_numbers = set()
    
    # Process each test
    print("Processing test data...")
    for test in data:
        test_type, test_number = extract_test_type_and_number(test['name'])
        if test_type is None:
            continue
        
        test_numbers.add(int(test_number))
        
        # Calculate cosine similarity between original and perturbed responses
        original_response = test['response_original']
        perturbed_response = test['response_perturb']
        orivsperturb = cosine_score(original_response, perturbed_response, model)
        
        # Format the key as "test_type #test_number"
        key = f"{test_type} #{test_number}"
        
        # Store original and perturbed scores for each test
        test_data[key] = {
            'original': test['score_original'],
            'perturbed': test['score_perturb'],
            'orivsperturb': orivsperturb
        }
    
    # Create CSV
    print("Writing to CSV file...")
    with open(csv_file, 'w', newline='') as f:
        writer = csv.writer(f)
        
        # Write header row
        writer.writerow(['', 'original', 'perturbed', 'orivsperturb'])
        
        # Use the specified order for test types
        ordered_types = [
            "taxonomy",
            "negation",
            "srl",
            "vocabulary",
            "fairness",
            "robustness",
            "logic",
            "coreference"
        ]
        
        # Sort test numbers
        sorted_numbers = sorted(test_numbers)
        
        # Write data rows for each test type and number combination
        for num in sorted_numbers:
            for test_type in ordered_types:
                key = f"{test_type} #{num}"
                if key in test_data:
                    writer.writerow([
                        key,
                        test_data[key]['original'],
                        test_data[key]['perturbed'],
                        test_data[key]['orivsperturb']
                    ])



### run script gpt

In [2]:
if __name__ == "__main__":
    convert_json_to_csv('gpt_std_zero.json', 'gpt_std_zero.csv')
    print("Conversion completed. CSV file created: gpt_std_zero.csv")

Loading SentenceTransformer model...
Processing test data...
Writing to CSV file...
Conversion completed. CSV file created: gpt_std_zero.csv


In [3]:
if __name__ == "__main__":
    convert_json_to_csv('gpt_std_one.json', 'gpt_std_one.csv')
    print("Conversion completed. CSV file created: gpt_std_one.csv")

Loading SentenceTransformer model...
Processing test data...
Writing to CSV file...
Conversion completed. CSV file created: gpt_std_one.csv


In [4]:
if __name__ == "__main__":
    convert_json_to_csv('gpt_std_few.json', 'gpt_std_few.csv')
    print("Conversion completed. CSV file created: gpt_std_few.csv")

Loading SentenceTransformer model...
Processing test data...
Writing to CSV file...
Conversion completed. CSV file created: gpt_std_few.csv


### run script gemini

In [5]:
if __name__ == "__main__":
    convert_json_to_csv('gemini_std_zero.json', 'gemini_std_zero.csv')
    print("Conversion completed. CSV file created: gemini_std_zero.csv")

Loading SentenceTransformer model...
Processing test data...
Writing to CSV file...
Conversion completed. CSV file created: gemini_std_zero.csv


In [6]:
if __name__ == "__main__":
    convert_json_to_csv('gemini_std_one.json', 'gemini_std_one.csv')
    print("Conversion completed. CSV file created: gemini_std_one.csv")

Loading SentenceTransformer model...
Processing test data...
Writing to CSV file...
Conversion completed. CSV file created: gemini_std_one.csv


In [7]:
if __name__ == "__main__":
    convert_json_to_csv('gemini_std_few.json', 'gemini_std_few.csv')
    print("Conversion completed. CSV file created: gemini_std_few.csv")

Loading SentenceTransformer model...
Processing test data...
Writing to CSV file...
Conversion completed. CSV file created: gemini_std_few.csv
